# Library 불러오기

In [0]:
!pip install geopandas  ## goepandas 설치

ERROR: Operation cancelled by user
^C


In [0]:
from  google.colab import drive    ## 구글 드라이브 마운트
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import networkx as nx            ## 네트워크 구성할 때 사용하는 library
import numpy as np               ## 여러가지 행렬계산에 용이한 library(배열)
import matplotlib.pyplot as plt  ## 그래프 그릴 때 사용
import geopandas as gpd          ## geography data 사용하는데 도움을 주는 library

In [0]:
!ls

drive  sample_data


# 네트워크 선언

In [0]:
Network = nx.DiGraph() #방향성 있느거 클래스 선언

# **DATA 불러오기**

In [0]:
name = 'Daegu'
path = '/content/drive/My Drive/korea_street_network_graph/'
link_list = gpd.read_file('/content/drive/My Drive/용성이형/korea_street_network/{0}_links_190709.csv'.format(name)) ## 링크 리스트
node_list = gpd.read_file('/content/drive/My Drive/용성이형/korea_street_network/{0}_nodes_190709.csv'.format(name)) ## 노드 리스트

노드와 링크 리스트 확인해보자. 

In [0]:
node_list

,field_1,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,USER_ID,WORKSTATE,DEPT_CODE,STNL_REG,TMPID,UPLOAD_ID,latitude,longitude,geometry
0,0,1500000100,104,수성교서편,0,,dgsi,5,27000,150,,,35.8617329941921,128.6084450139247,None
1,1,1500000200,101,수성교네거리,1,,dgsi,5,27000,150,,,35.861892993242414,128.60820701355163,None
2,2,1500000300,101,-,1,,dgsi,5,27000,150,,,35.86266299405892,128.5741370130592,None
3,3,1500000400,101,삼덕네거리,1,,dgsi,5,27000,150,,,35.86324199382722,128.6026260132258,None
4,4,1500000500,101,봉산육거리,1,,dgsi,5,27000,150,,,35.86389599409578,128.59945901289174,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1878,1878,3910020200,105,-,0,,dgsi,5,27000,391,,,35.64222699358768,128.4590870131735,None
1879,1879,3910020300,105,경남창녕-대구달성 경계,0,,dgsi,5,27000,391,,,35.636140993386,128.4890630135809,None
1880,1880,3910071100,105,대구-창녕군경계,0,,mappers,5,60190,391,TMPID20160820054,mappers20161007133943,35.637074502783435,128.45393358918645,None
1881,1881,3910071500,101,대견교차로,0,,mappers,5,60190,391,TMPID20160820104,mappers20161007133943,35.63744429866433,128.46254173614713,None


In [0]:
link_list

,field_1,LINK_ID,F_NODE,T_NODE,ROAD_USE,LANES,ROAD_RANK,ROAD_NAME,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,REMARK,USER_ID,WORKSTATE,DEPT_CODE,STNL_REG,ROAD_TYPE,ROAD_NO,TMPID,UPLOAD_ID,SOSFNODEID,SOSTNODEID,SHAPE_STLe,geometry
0,0,1500026100,1500002300,1500007300,0,2.0,104,동덕로,0,000,60.0,0,0.0,0.0,,,5,27000,150,000,67,RO2700078900000,,,RN2700026600000,322.948001975,None
1,1,1500026200,1500007300,1500001100,0,2.0,104,동덕로,0,000,60.0,0,0.0,0.0,,,5,27000,150,000,67,RO2700079000000,,RN2700026600000,,348.261953915,None
2,2,1500026300,1500001100,1500007300,0,2.0,104,동덕로,0,000,60.0,0,0.0,0.0,,,5,27000,150,000,67,RO2700079100000,,,RN2700026600000,358.579832351,None
3,3,1500026400,1500007300,1500002300,0,2.0,104,동덕로,0,000,60.0,0,0.0,0.0,,,5,27000,150,000,67,RO2700079200000,,RN2700026600000,,324.96833707,None
4,4,1500026500,1500000400,1500007400,0,5.0,104,달구벌대로,0,000,70.0,0,0.0,0.0,,,5,27000,150,000,40,RO2700079300000,,,RN2700024300000,318.266728548,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5737,5737,1570686600,1570228200,1570005300,0,2.0,103,일반국도5호선,1,000,70.0,0,0.0,500.0,,geostoryco,5,60170,157,000,5,TMPID20180821212808,geostoryco20180829141113,TMPID20180814154567,,2122.32379899,None
5738,5738,3670014304,3670006601,1570005600,0,3.0,101,광주대구고속도로,0,000,100.0,5,0.0,0.0,기존 링크 분할 수정,geostoryco,5,60170,157,000,12,,geostoryco20180918111311,,,8606.18661634,None
5739,5739,1570030100,3910015900,1570012600,0,2.0,101,중부내륙고속도로,0,000,100.0,5,0.0,0.0,,geostoryco,5,60170,157,000,14,,geostoryco20180918111311,,,15213.0571736,None
5740,5740,1570030500,1570005600,1570005900,0,1.0,101,구마고속도로진입로,0,101,50.0,0,0.0,0.0,,nodelinkmanager,5,50000,157,000,12,RO2700001500000,nodelinkmanager2019011613,,,637.820683505,None


## 불러온 노드와 링크 data로 **네트워크 구축**

In [0]:
## 네트워크 클래스
street_network = nx.DiGraph()
## 노드 추가
for l, e in node_list.iterrows():
    street_network.add_node(int(e['NODE_ID']), POS = (float(e['longitude']), float(e['latitude'])))
## 링크 추가
for l, e in link_list.iterrows():
    street_network.add_edge(int(e['F_NODE']),int(e['T_NODE']),LINK_ID = int(e['field_1']), length = float(e['SHAPE_STLe']), LANES = float(e['LANES']))


## remove wrong data nodes
bad_data = list(street_network.nodes())[0]
if bad_data == 1300000100: street_network.remove_node(1450000800) ## Busan wrong data
elif bad_data == 1830000100: street_network.remove_nodes([1865789300, 1865791600]) ## Daejeon wrong data 
elif bad_data == 1500000100: street_network.remove_node(1550007800) ## Daegu wrong data

## 노드,링크 수 확인

In [0]:
print('노드수:',len(street_network.nodes))
print('링크수:',len(street_network.edges))

노드수: 1883
링크수: 5732


In [159]:
import time
start_time=time.time()
BC = nx.betweenness_centrality(street_network)
print(time.time()-start_time)

17.76520800590515


In [160]:
18*188

3384

In [161]:
3384/60

56.4

**각 노드들의 BC를 계산 해서 그래프를 그릴건데**, 
**BC가 높은 순서부터** 하나하나씩 제거해 나가면서 BC를 계산 할 것이다. 
언제까지?? **전체 노드의 10%가 없어 질 때 까지** 하나하씩. 
계산된 BC그래프로** PDF 그래프를 그리고**  **기울기**를 구할 건데, 그건 위 과정에서 0,2,4,6,8,10 % 없어졌을 때만 구할거다. 해보자 

# **appendix**
밑에 appendix(a), appendix(b)는 필요한 함수를 미리 정의 해놓은 것이다 .일단 무시

In [0]:
#appendix (a)  BC distribution 그리는 함수 
def pdf(BC_dist, remove_rate, city, bins_size=1000):
    """plot & save BC dist and exponent"""
    hist_BC,bins_BC=np.histogram(list(BC_dist.values()),bins_size)        ## 히스토그램
    hist_re = hist_BC[np.where(hist_BC>0)]
    x_hist = bins_BC[1:bins_size+1][np.where(hist_BC>0)]
    ## 선형회귀 적용할 최소, 최대 범위
    min_BC,max_BC = min(x_hist),min(x_hist)*50  
    x_hist =  x_hist[x_hist<=max_BC]
    x_hist = x_hist[x_hist>=min_BC]
    hist_re=hist_re[:len(x_hist)]
    linear_hist=[list(x_hist),list(hist_re)]

    a,b=linearReg(np.log10(np.array(linear_hist).T[1:]))
    plt.plot([10**(np.log10(x_hist[1])),10**(np.log10(x_hist[-1]))],[10**((np.log10(x_hist[1]))*a+b),10**(np.log10(x_hist[-1])*a+b)],'r:')
    plt.scatter(bins_BC[1:bins_size+1],hist_BC,c='olive',alpha=0.7)    ## 그래프 그리기
    plt.xscale('log')                                    ## x 스케일 : log
    plt.yscale('log')                                    ## y 스케일 : log
    plt.xlim(bins_BC[1]*0.2,bins_BC[-1]*5)                                   ## 그래프의 x축 범위
    plt.ylim(hist_BC[-1]*0.2,hist_BC[1]*5)                                   ## 그래프의 y축 범위
    plt.xlabel(r'$c_B$')                                 ## x축 라벨
    plt.ylabel('PDF')                                    ## y축 라벨
    plt.tight_layout                                     ## 그림, 글자 겹치지 않게 해주는 명령어
    plt.show()
    #plt.savefig(path + '{}_BC_PDF_removeRate_{}_exponent_{}.png'.format(city, remove_rate, round(-a,3)),dpi=300, transparent=True)
    #plt.savefig('/content/drive/My Drive/용성이형/Seoul_net',dpi=300)
    
    #plt.close()


In [0]:
#appendix(b) 추세선 그리기위한 함수(선형회귀용함수)
def qr_householder(A):
    m, n = A.shape
    Q = np.eye(m) # Orthogonal transform so far
    R = A.copy() # Transformed matrix so far

    for j in range(n):
        # Find H = I - beta*u*u' to put zeros below R[j,j]
        x = R[j:, j]
        normx = np.linalg.norm(x)
        rho = -np.sign(x[0])
        u1 = x[0] - rho * normx
        u = x / u1
        u[0] = 1
        beta = -rho * u1 / normx

        R[j:, :] = R[j:, :] - beta * np.outer(u, u).dot(R[j:, :])
        Q[:, j:] = Q[:, j:] - beta * Q[:, j:].dot(np.outer(u, u))
    return Q, R
def linearReg(A):
    m, n = A.shape
    AA = np.array([A[:,0], np.ones(m)]).T
    b = A[:, 1]

    Q, R = qr_householder(AA)
    b_hat = Q.T.dot(b)

    R_upper = R[:n, :]
    b_upper = b_hat[:n]

    x = np.linalg.solve(R_upper, b_upper)
    slope, intercept = x
    return slope, intercept 

노드 3267개의 2%는 65개다, 총 65*5= 325개의 노드를 하나하나 씩 빼면서 BC를 구할 것인데, 노드를 65개 뺄 때마다 (2%씩 뺄 때마다) BC distribution을 그리고, 기울기를 구할 것이다. 


In [0]:
one_percent_node=int(len(street_network.nodes)*0.01)

#for i in range(one_percent_node*10):  # (도시의) 노드 개수의 10 %만큼 실행한다.
for i in range(5):
  BC = nx.betweenness_centrality(street_network) #BC계산 

  bc_value=np.zeros(len(street_network.nodes)) #도시의 노드 수 만큼의 0을 가지고 있는 array를 만든다
  n=0
  for i in street_network.nodes :
    if n==len(street_network.nodes): break
    bc_value[n]=BC[i] 
    n+=1 #n을 1씩 늘려가며 0으로 가득찬 bc_value 방들에 계산한 BC를 넣어준다

  node_list['BC']=bc_value #위에 pandas에 'BC'라는 항목을 추가해주고 그안에 최신화한 bc_value array를 넣어준다
  node_list=node_list.sort_values('BC',ascending=False) #pandas에 저장된 BC값들을 내림차순으로 정렬한다(높은 BC를 가진 NODE순으로 정렬)

  highest_BC_node=int(node_list.values.tolist()[0][1]) # 정렬된 노드리스트를 list로 바꾼 후, 거기서 가장 첫번째 줄의 두번째 값인 NODE_ID를 정의한다 
  highest_BC_node_label = int(node_list.values.tolist()[0][0])
  node_list = node_list.drop(highest_BC_node_label)
  street_network.remove_node(highest_BC_node) #그 NODE ID를 제거한다 

#위의 과정을 "(도시의) 노드 개수의 10 %" 번  반복할 것인데, 2% 제거할 때 마다 BC distrtibution 그리고 기울기 구하기로 했지
  #pdf(nodes_left) 라는 새롭게 정의했다. appendix (a)
  
  
# if i in np.linspace(one_percent_node*2, one_percent_node*10, 5):
  print(i)
  if i==4:
     pdf(BC, 1, city)
 
 

  

0
1
2
3
4


NameError: ignored